In [1]:
library(OmnipathR)
# library(nichenetr)
library(tidyverse)
library(dplyr)
library(VennDiagram)
library(ggplot2)
library(utils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: grid

Loading required package: futile.logger



```import_ligrecextra_interactions```

link: to documentation <https://r.omnipathdb.org/reference/import_ligrecextra_interactions.html>

This LR dataset contains ligand-receptor interactions without literature reference. The ligand-receptor interactions supported by literature references are part of the `omnipath` dataset.


Default params(without filtering resources) has 8350 edges. The table includes columns as follows. 

'source' 'target' 'source_genesymbol' 'target_genesymbol' 'is_directed' 'is_stimulation' 'is_inhibition' 'consensus_direction' 'consensus_stimulation' 'consensus_inhibition' 'sources' 'references' 'curation_effort' 'n_references' 'n_resources'

**The consensus score is if resources supporting the classification of an entity into a category based on combined information of many resources.**

<span style="color:red">I do not undertand how it can have sources but no references.</span>

| sources                                                         | ref | cur_effort | n_ref | n_source |
|-----------------------------------------------------------------|-----|------------|-------|----------|
|      Baccin2019;CellCall;PhosphoPoint;Ramilowski2015_Baccin2019 | NA  | 0          | 0     | 3        |
| Baccin2019;CellCall;PhosphoPoint;Ramilowski2015_Baccin2019;Wang | NA  | 0          | 0     | 4        |
|                                                                 |     |            |       |          |

In [2]:
# https://r.omnipathdb.org/reference/curated_ligand_receptor_interactions.html
# curated=curated_ligand_receptor_interactions()
lr <- import_ligrecextra_interactions()

In [3]:
lr <- lr %>% filter(!duplicated(lr[, c("source_genesymbol", "target_genesymbol")]))

```import_omnipath_intercell``` Imports the OmniPath intercellular **communication role annotation** database. It provides information on the roles in inter-cellular signaling. E.g. if a protein is a ligand, a receptor, an extracellular matrix (ECM) component, etc.

In [4]:
# the genesymbol PIK3CD-AS1 is causing an error in the later steps, we convert the name
# lr[lr == "PIK3CD-AS1"] <- "PIK3CD"

In [5]:
anno_raw <- import_omnipath_intercell()
#subset annotation DB to only ligand and receptors
anno_lig <- anno_raw %>%
    dplyr::filter(category %in% c("receptor","ligand"))
# Drop rows where the values in the "parent", "database", and "uniprot" columns are duplicated
anno_raw <- anno_raw %>% filter(!duplicated(anno_raw[, c("parent", "database", "uniprot")]))

# Breaking down complexes

In [6]:
#filter only those are in complex
complex <- filter(lr, grepl('COMPLEX', target) | grepl('COMPLEX',source))
complex$source <- sub("COMPLEX:", "", complex$source)
complex$target <- sub("COMPLEX:", "", complex$target)

In [7]:
#complexes are seperated into individual components
components_target <- unique(unlist(strsplit(complex$source_genesymbol,"_")))
components_source <- unique(unlist(strsplit(complex$target_genesymbol,"_")))
components_both <- c(components_target,components_source)
components_both <- unique(components_both)

Below, we produce all the the possible pairs. 

Example: lets assume complex G1_G2_G3 is linked to another complex G4_G5_G6

| c1 | c2 | complex_origin    |
|----|----|-------------------|
| G1 | G2 | G1_G2_G3_G4_G5_G6 |
| G1 | G3 | G1_G2_G3_G4_G5_G6 |
| G1 | G4 | G1_G2_G3_G4_G5_G6 |
| G1 | G5 | G1_G2_G3_G4_G5_G6 |
| G1 | G6 | G1_G2_G3_G4_G5_G6 |
| G2 | G1 | G1_G2_G3_G4_G5_G6 |
| G2 | G3 | G1_G2_G3_G4_G5_G6 |
| .. | .. | G1_G2_G3_G4_G5_G6 |

In [8]:
# Produce all the possbile pairwise pairs

results <- list()

# Loop through each row of the data frame
for (i in 1:nrow(complex)) {
  # Extract the values from the first column
  values1 <- unlist(strsplit(as.character(complex[i, "source_genesymbol"]), "_"))
  # Extract the values from the second column
  values2 <- unlist(strsplit(as.character(complex[i, "target_genesymbol"]), "_"))
  # Keep the original pair
  original <- paste(complex[i, "source_genesymbol"],complex[i, "target_genesymbol"],sep="_")
  # Generate all the pairwise combinations using combn
  pairs <- combn(c(values1, values2), 2)
  pairs <- t(pairs)
  pairs <- cbind(pairs,original)
  # Append the results to the list
  results[[i]] <- as.data.frame(pairs)
    colnames(results[[i]]) = c("source","target","complex_pair")
    row.names(results[[i]]) <- NULL
}

# Bind the results into a single data frame
result_df2 <- as.data.frame(do.call(rbind, results))

# Switch the values in the "col1" and "col2" columns
df1 <- cbind(result_df2[,2], result_df2[,1], result_df2[,3])
colnames(df1) <- names(result_df2)
# Bind the rows into a single data frame
result_df <- rbind(result_df2, df1)


# Drop the self links
result_df <- result_df %>% filter(!duplicated(result_df[, c("source", "target")]))

# View the resulting data frame
str(result_df)

'data.frame':	3272 obs. of  3 variables:
 $ source      : chr  "IL17A" "IL17A" "IL17RA" "NPNT" ...
 $ target      : chr  "IL17RA" "IL17RC" "IL17RC" "ITGA8" ...
 $ complex_pair: chr  "IL17A_IL17RA_IL17RC" "IL17A_IL17RA_IL17RC" "IL17A_IL17RA_IL17RC" "NPNT_ITGA8_ITGB1" ...


# Annotation of components

The complexes are decomposed into their individual components. The Omnipath Intercell annotation database is imported and used to annotate each component. If at least two databases categorize a component as a ligand or receptor, it is annotated as such. If not, we check other possible categories such as extracellular matrix, secreted, and transmembrane 



In [9]:
#create a df to store annotation
df <- data.frame(genesymbol = character(length(components_both)), score = numeric(length(components_both)), parent = character(length(components_both)), stringsAsFactors = FALSE)

In [10]:
# Check if the components are categorized as ligands or receptors

for (x in 1:length(components_both)) {
#     maxvalue=max(filter(anno, uniprot==components[x])$consensus_score)
    genename <- components_both[x]
    parent_score <- sort(table(filter(anno_lig, genesymbol==components_both[x])$parent), decreasing = T, na.last = T)[1]
    parent_category <- names(parent_score)
    
    if (is.null(parent_category)) {
      parent_category <- "NA"
      parent_score <- 0
    }
    
    df[x, "genesymbol"] <- genename
    df[x, "score"] <- parent_score
    df[x, "parent"] <- parent_category

#     df$genesymbol[x] <- genename
#     df$score[x] <- parent_score
#     df$parent[x] <- parent_category
}

table(df$parent)


  ligand       NA receptor 
     238       46      198 

In [11]:
# If a component is not classified as a ligand or receptor, we may consider other categories such as 
# extracellular matrix, secreted, and transmembrane.# annotated others such as secreted, ecm etc

df_na <- filter(df, parent=="NA")$genesymbol

for (x in 1:length(df_na)) {
#     maxvalue=max(filter(anno, uniprot==components[x])$consensus_score)
    genesymbol <- df_na[x]
    parent_score <- sort(table(filter(anno_raw, genesymbol==df_na[x])$parent), decreasing = T, na.last = T)[1]
    parent_category <- names(parent_score)

    df <- df %>% mutate(parent = ifelse(genesymbol == df_na[x], parent_category, parent))
    df <- df %>% mutate(score = ifelse(genesymbol == df_na[x], parent_score, score))

}

table(df$parent)


          ecm        ligand      receptor      secreted transmembrane 
           29           238           198             8             9 

In [12]:
# categorize ecm/secreted as ligand
df$parent <- replace(df$parent, df$parent == "ecm", "ligand")
df$parent <- replace(df$parent, df$parent == "secreted", "ligand")

# Linking 1

We are using the Omnipath intercellular interaction network, which is the largest available network of its kind, to detect interactions rather than make predictions. The creators of the network have noted that it may contain a large number of false positives. Despite this, we are using it in combination with an annotations database to detect interactions. The network has a size of 98,165 edges.

In [13]:
# Import All post-translational interactions
pt <- import_post_translational_interactions()

In [14]:
# "Separate the annotated components of complexes based on their type."
ligands <- filter(df, parent=="ligand")
receptors <- filter(df, parent=="receptor")

In [15]:
# Filter the PT network to include only the components of the complexes
pt <- pt %>%
    dplyr::filter(source_genesymbol %in% ligands$genesymbol) %>%
    dplyr::filter(target_genesymbol %in% receptors$genesymbol) %>%
    dplyr::distinct()

In [16]:
# remove duplicated
pt <- pt %>% filter(!duplicated(pt[, c("source_genesymbol", "target_genesymbol")]))

In [17]:
# create the pairs
pt$pair=paste(pt$source_genesymbol, pt$target_genesymbol,sep="_")
result_df$pair=paste(result_df$source, result_df$target,sep="_")

In [18]:
# The data frame result_df consists of all the pairwise pair combinations, 
# and we are checking if those pairs exist in the PT network
pt_edges <- result_df %>%
    filter(pair %in% pt$pair)

str(pt_edges)

'data.frame':	983 obs. of  4 variables:
 $ source      : chr  "IL17A" "IL17A" "NPNT" "NPNT" ...
 $ target      : chr  "IL17RA" "IL17RC" "ITGA8" "ITGB1" ...
 $ complex_pair: chr  "IL17A_IL17RA_IL17RC" "IL17A_IL17RA_IL17RC" "NPNT_ITGA8_ITGB1" "NPNT_ITGA8_ITGB1" ...
 $ pair        : chr  "IL17A_IL17RA" "IL17A_IL17RC" "NPNT_ITGA8" "NPNT_ITGB1" ...


In [19]:
filter(result_df, complex_pair=="IL17A_IL17RA_IL17RC")

source,target,complex_pair,pair
<chr>,<chr>,<chr>,<chr>
IL17A,IL17RA,IL17A_IL17RA_IL17RC,IL17A_IL17RA
IL17A,IL17RC,IL17A_IL17RA_IL17RC,IL17A_IL17RC
IL17RA,IL17RC,IL17A_IL17RA_IL17RC,IL17RA_IL17RC
IL17RA,IL17A,IL17A_IL17RA_IL17RC,IL17RA_IL17A
IL17RC,IL17A,IL17A_IL17RA_IL17RC,IL17RC_IL17A
IL17RC,IL17RA,IL17A_IL17RA_IL17RC,IL17RC_IL17RA


In [20]:
filter(pt_edges, complex_pair=="IL17A_IL17RA_IL17RC")

source,target,complex_pair,pair
<chr>,<chr>,<chr>,<chr>
IL17A,IL17RA,IL17A_IL17RA_IL17RC,IL17A_IL17RA
IL17A,IL17RC,IL17A_IL17RA_IL17RC,IL17A_IL17RC


# Complexes are broken down, now we can combine with the rest of the db

In [21]:
single_components = filter(lr, !grepl('COMPLEX', target) & !grepl('COMPLEX',source))

In [22]:
single_components <- single_components %>%
  select(source_genesymbol, target_genesymbol) %>%
  rename(source=source_genesymbol, target=target_genesymbol) %>%
  mutate(complex_pair = NA)

In [23]:
single_components$pair <- paste(single_components$source, single_components$target, sep="_")

In [24]:
#merge the single ones, with complexes componenets that are detected via PT_DB
complete <- rbind(single_components, pt_edges)

In [25]:
#remove the duplicated ones, and drop the last ones, which are coming from the complexes
complete <- complete[ !duplicated(complete[, "pair"], fromLast=F),]

# Protein Descriptions

We use mygene library to get the protein descriptions

In [26]:
library(mygene)

Loading required package: GenomicFeatures

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from ‘package:tidyr’:

    expand



In [27]:
# get gene symbols
gene_symbols <- unique(c(complete$source,complete$target))

In [28]:
prot_descriptions <- queryMany(gene_symbols, scopes = "symbol", 
                              fields = c("name"), 
                              species = "human", as_dataframe = "True")

Querying chunk 1

Querying chunk 2

Querying chunk 3



Finished
Pass returnall=TRUE to return lists of duplicate or missing query terms.


In [29]:
prot_descriptions <- as.data.frame(prot_descriptions)

In [30]:
#map protein descriptions to complete set

for (x in 1:nrow(complete)) {
    ligand_symbol=complete[x,]$source
    receptor_symbol=complete[x,]$target
    ligand_description=filter(prot_descriptions, query==ligand_symbol)$name
    receptor_description=filter(prot_descriptions, query==receptor_symbol)$name
    lig_id=filter(anno_raw, genesymbol==ligand_symbol)$uniprot[1]
    rec_id=filter(anno_raw, genesymbol==receptor_symbol)$uniprot[1]
    
    if (ligand_symbol=="PIK3CD-AS1") {
      lig_id <- "O00329"
    }
    
#     if (is.null(receptor_description)) {
#       receptor_description <- "NA"
#     }
    

    complete[x, "ligand.name"] = ligand_description[1]
    complete[x, "receptor.name"] = receptor_description[1]
    complete[x, "partner_a"] = lig_id
    complete[x, "partner_b"] = rec_id
}

In [31]:
head(complete)

source,target,complex_pair,pair,ligand.name,receptor.name,partner_a,partner_b
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CALM1,TRPC3,NA,CALM1_TRPC3,calmodulin 1,transient receptor potential cation channel subfamily C member 3,P0DP23,Q13507
S100A10,TRPV6,NA,S100A10_TRPV6,S100 calcium binding protein A10,transient receptor potential cation channel subfamily V member 6,P60903,Q9H1D0
JAK2,EPOR,NA,JAK2_EPOR,Janus kinase 2,erythropoietin receptor,O60674,P19235
NOTCH1,JAG2,NA,NOTCH1_JAG2,notch receptor 1,jagged canonical Notch ligand 2,P46531,Q9Y219
JAG2,NOTCH1,NA,JAG2_NOTCH1,jagged canonical Notch ligand 2,notch receptor 1,Q9Y219,P46531
DLL1,NOTCH1,NA,DLL1_NOTCH1,delta like canonical Notch ligand 1,notch receptor 1,O00548,P46531


In [32]:
#reorder columns
complete <- complete[, c("pair", "source", "ligand.name", "target", "receptor.name", "complex_pair",
                         "partner_a","partner_b")]
#rename column names
names(complete) <- c("Pair.Name", "Ligand", "Ligand.Name", "Receptor", "Receptor.Name", "complex_pair",
                    "partner_a","partner_b")

In [33]:
head(complete)

Pair.Name,Ligand,Ligand.Name,Receptor,Receptor.Name,complex_pair,partner_a,partner_b
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CALM1_TRPC3,CALM1,calmodulin 1,TRPC3,transient receptor potential cation channel subfamily C member 3,NA,P0DP23,Q13507
S100A10_TRPV6,S100A10,S100 calcium binding protein A10,TRPV6,transient receptor potential cation channel subfamily V member 6,NA,P60903,Q9H1D0
JAK2_EPOR,JAK2,Janus kinase 2,EPOR,erythropoietin receptor,NA,O60674,P19235
NOTCH1_JAG2,NOTCH1,notch receptor 1,JAG2,jagged canonical Notch ligand 2,NA,P46531,Q9Y219
JAG2_NOTCH1,JAG2,jagged canonical Notch ligand 2,NOTCH1,notch receptor 1,NA,Q9Y219,P46531
DLL1_NOTCH1,DLL1,delta like canonical Notch ligand 1,NOTCH1,notch receptor 1,NA,O00548,P46531


In [34]:
# filter(previous_db, Receptor=="NOTCH1")

# append the original structure from OmniPath

In [35]:
lr$pair <- paste(lr$source_genesymbol, lr$target_genesymbol, sep="_")

Create a column to merge with. We are doing this because the complex pairs in our data have been broken down, while they are not broken down in the original data. The new column will allow us to match and merge the broken-down pairs with the corresponding pairs in the original data

In [36]:
complete <- complete %>% mutate(to_merge = ifelse(!is.na(complex_pair), complex_pair, paste(Ligand, Receptor, sep="_")))

In [37]:
concatanated <- as.data.frame(merge(complete, lr, by.x = "to_merge", by.y = "pair"))

In [38]:
nrow(complete) == nrow(concatanated)

[1] TRUE

In [39]:
concatanated <- concatanated %>% dplyr::select(-to_merge)

# tagging curated ones

In [41]:
curated <- curated_ligand_receptor_interactions()
curated <- curated %>% filter(!duplicated(curated[, c("source_genesymbol", "target_genesymbol")]))
curated$pair <- paste(curated$source_genesymbol, curated$target_genesymbol, sep="_")

In [43]:
for (x in 1:nrow(concatanated)) {
    
    pair=concatanated[x,]$Pair.Name
    complex_pair=concatanated[x,]$Pair.Name
    ligand_symbol=concatanated[x,]$complex_pair
    
    
    if (pair %in% curated$pair || complex_pair %in% curated$pair) {
      concatanated[x, "curated"] = TRUE
    }
    else {
        concatanated[x, "curated"] = FALSE
    }
    
#     if (is.null(receptor_description)) {
#       receptor_description <- "NA"
#     }
    }

In [52]:
#this column is needed when building CellPhoneDB
concatanated$annotation_strategy <- ifelse(concatanated$curated == TRUE, "OmniPath_curated", "OmniPath")

In [53]:
write.csv(concatanated, "L_R_OmniPathFull.csv", row.names=FALSE)

In [48]:
filter(concatanated,Ligand=="PIK3CD-AS1")

Pair.Name,Ligand,Ligand.Name,Receptor,Receptor.Name,complex_pair,partner_a,partner_b,source,target,⋯,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,sources,references,curation_effort,n_references,n_resources,curated
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<int>,<lgl>
PIK3CD-AS1_LY6G6C,PIK3CD-AS1,PIK3CD antisense RNA 1,LY6G6C,lymphocyte antigen 6 family member G6C,NA,O00329,O95867,Q5SR53,O95867,⋯,0,0,0,0,Fantom5_LRdb;LRdb;iTALK,NA,0,0,2,FALSE
PIK3CD-AS1_SLC16A4,PIK3CD-AS1,PIK3CD antisense RNA 1,SLC16A4,solute carrier family 16 member 4,NA,O00329,O15374,Q5SR53,O15374,⋯,0,0,0,0,Fantom5_LRdb;LRdb;iTALK,NA,0,0,2,FALSE


In [ ]:
# concatanated[concatanated == "PIK3CD-AS1"] <- "PIK3CD"

In [ ]:
# lr[lr == "PIK3CD-AS1"] <- "PIK3CD"